## Teste 1
##### Programação grosseira em busca dos melhores dados e dos melhores modelos

In [387]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import metrics
from warnings import simplefilter

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

plt.rcParams['figure.figsize'] = (25, 8)
#sns.set_style('whitegrid')
sns.set_style('white')
sns.set_context('poster')

In [388]:
file = 'datasets/dataset_cler.csv'
file = 'C:/Users/André/AppData/Roaming/MetaQuotes/Tester/D0E8209F77C8CF37AD8BF550E51FF075/Agent-127.0.0.1-3000/MQL5/Files/dataset_cler.csv'

df = pd.read_csv(file, sep=";", encoding='utf-16')
del df['Ref']
df.shape

(1887, 258)

In [389]:
df.head(2)

,ampl_9,desv_amp_9,body_9,body_per_9,pavio_sup_9,pavio_sup_per_9,pavio_inf_9,pavio_inf_per_9,high_dist_9,low_dist_9,...,pavio_inf_h2_1,pavio_inf_per_h2_1,high_dist_h2_1,low_dist_h2_1,type_h2_1,atr,qt_bars,oper,take,status
0,145.0,1.07,65.0,44.83,5.0,3.45,75.0,51.72,40.0,50.0,...,0.0,0.00,20.0,280.0,down,91.79,8,Sell,-40.0,Loss
1,150.0,1.11,65.0,43.33,35.0,23.33,50.0,33.33,25.0,40.0,...,215.0,47.78,615.0,620.0,up,88.21,4,Sell,-40.0,Loss


In [390]:
df['status'].value_counts()

Loss    949
Gain    938
Name: status, dtype: int64

In [391]:
df.drop(df.loc[(df['status']!='Loss') & (df['status']!='Gain')].index.values, inplace=True)

In [392]:
df['status'].value_counts()

Loss    949
Gain    938
Name: status, dtype: int64

In [393]:
#train_features = df.columns.difference(['take', 'status', 'oper'])
X = df.drop(['take','status','oper'], axis=1) #df[train_features]
y = df['status']

x_columns = X.columns

In [394]:
index_features_category = []
for index, col in enumerate(X.columns):
    if X.dtypes[index]=='object':
        index_features_category.append(index)
index_features_category[:10]

[10, 21, 32, 43, 54, 65, 76, 87, 98, 109]

In [395]:
encoder = LabelEncoder()
encoder.fit(['inside', 'up', 'down'])
columns_categorical = X.select_dtypes(include=['object']).columns
for col_cat in columns_categorical:
    X[col_cat] = encoder.transform(X[col_cat])

ohe = OneHotEncoder(categorical_features=index_features_category)
X = ohe.fit_transform(X).toarray()

In [396]:
def fit(X_train, X_test, y_train, y_test, model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
    return y_pred 

def fitAll(X_train, X_test, y_train, y_test, algorithms):
    similaridade = []
    for item in algorithms:
        name = item[0]
        algorithm = item[1]
        
        algorithm.fit(X_train, y_train)
        y_pred = algorithm.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        #print(name, ' => ', accuracy)
        similaridade.append((accuracy, name))
        
    similaridade.sort()
    similaridade.reverse()
    
    for j in similaridade:
        print(j)

def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()

In [397]:
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

standar = StandardScaler()
standar.fit(X)
X = standar.fit_transform(X)

In [398]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=True)

In [399]:
algorithms = []

algorithms.append(['DecisionTreeClassifier', DecisionTreeClassifier(random_state=0)])
algorithms.append(['RandomForestClassifier', RandomForestClassifier(random_state=0)])
algorithms.append(['GaussianNB', GaussianNB()])
algorithms.append(['RidgeClassifier', RidgeClassifier()])
algorithms.append(['MLPClassifier', MLPClassifier()])
algorithms.append(['GradientBoostingClassifier', GradientBoostingClassifier()])
algorithms.append(['SVC', SVC(gamma='auto')])
algorithms.append(['KNeighborsClassifier', KNeighborsClassifier()])
algorithms.append(['LogisticRegression', LogisticRegression()])

fitAll(X_train, X_test, y_train, y_test, algorithms)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(0.527336860670194, 'RidgeClassifier')
(0.5220458553791887, 'LogisticRegression')
(0.5185185185185185, 'MLPClassifier')
(0.5167548500881834, 'GradientBoostingClassifier')
(0.5114638447971781, 'SVC')
(0.5061728395061729, 'RandomForestClassifier')
(0.5044091710758377, 'KNeighborsClassifier')
(0.48677248677248675, 'GaussianNB')
(0.4567901234567901, 'DecisionTreeClassifier')


test = GradientBoostingClassifier(learning_rate=0.1, n_estimators=1000)
y_pred = fit(X_train, X_test, y_train, y_test, test)

conf_matrix = confusion_matrix(y_test, y_pred)
conf_matrix

In [400]:
y_test[300:301]

1546    Gain
Name: status, dtype: object

In [401]:
X_test[300]

array([0.25984252, 0.09448819, 0.1031746 , 0.3611    , 0.20512821,
       0.23359966, 0.19736842, 0.43983534, 0.04265403, 0.09027778,
       1.        , 0.22641509, 0.05857741, 0.17021277, 0.5926    ,
       0.10714286, 0.12344444, 0.18181818, 0.30650667, 0.01945525,
       0.01476015, 0.        , 0.14953271, 0.05606524, 0.11538462,
       0.45      , 0.18181818, 0.53573342, 0.0212766 , 0.05249895,
       0.00378788, 0.03463203, 1.        , 0.05113636, 0.04442037,
       0.02027027, 0.25      , 0.12244898, 0.53844497, 0.08333333,
       0.26922249, 0.03184713, 0.024     , 0.        , 0.12621359,
       0.15479452, 0.02469136, 0.125     , 0.16666667, 0.55205918,
       0.15384615, 0.41249588, 0.0078125 , 0.02054795, 1.        ,
       0.09937888, 0.08650519, 0.0625    , 0.4211    , 0.03389831,
       0.11846102, 0.07142857, 0.53683137, 0.02955665, 0.04812834,
       0.        , 0.16197183, 0.14646465, 0.03174603, 0.1538    ,
       0.4375    , 0.86542377, 0.01694915, 0.04375   , 0.10344